## Day 49 Lecture 1 Assignment

In this assignment, we will apply GMM (Gaussian Mixture Modeling) clustering to a dataset containing player-season statistics for NBA players from the past four years.

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.special import entr

This dataset contains player-season statistics for NBA players from the past four years. Each row in this dataset represents a player's per-game averages for a single season. 

This dataset contains the following variables:

- Seas: season ('2019' = 2018-2019 season, '2018' = 2017-2018 season, etc.)
- Player: player name
- Pos: position
- Age: age
- Tm: team
- G: games played
- GS: games started
- MP: minutes played
- FG: field goals
- FGA: field goals attempted
- FG%: field goal percentage
- 3P: 3 pointers
- 3PA: 3 pointers attempted
- 3P%: 3 point percentage
- 2P: 2 pointers
- 2PA: 2 pointers attempted
- 2P%: 2 point percentage
- eFG%: effective field goal percentage
- FT: free throws
- FTA: free throws attempted
- FT%: free throw percentage
- ORB: offensive rebound
- DRB: defensive rebound
- TRB: total rebounds
- AST: assists
- STL: steals
- BLK: blocks
- TOV: turnovers
- PF: personal fouls
- PTS: points

Load the dataset.

In [2]:
# answer goes here
nba = pd.read_csv('nba_player_seasons.csv')
nba.head()

,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,2019,Álex Abrines,SG,25,OKC,31,2,19.0,1.8,5.1,...,0.923,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3
1,2019,Quincy Acy,PF,28,PHO,10,0,12.3,0.4,1.8,...,0.700,0.3,2.2,2.5,0.8,0.1,0.4,0.4,2.4,1.7
2,2019,Jaylen Adams,PG,22,ATL,34,1,12.6,1.1,3.2,...,0.778,0.3,1.4,1.8,1.9,0.4,0.1,0.8,1.3,3.2
3,2019,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,...,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,2019,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,...,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9


The goal is to cluster these player-seasons to identify potential player "archetypes".  
The pre-processing steps will be identical to what we previously did for K-means.

Begin by removing players whose season did not meet one of the following criteria:
1. Started at least 20 games
2. Averaged at least 10 minutes per game

In [3]:
# answer goes here
nba = nba[nba["GS"] >= 20]
nba = nba[nba["MP"] >= 10]
nba_og = nba.copy()


Choose a subset of numeric columns that is interesting to you from an "archetypal" standpoint. 

We will choose the following basic statistics: **points, total rebounds, assists, steals, blocks**, and **turnovers**, but you should feel free to choose other reasonable feature sets if you like. Be careful not to include too many dimensions (curse of dimensionality).

In [4]:
# answer goes here
cols = ["PTS", "TRB", "AST", "STL", "BLK", "TOV"]
nba = nba[cols]
nba.head()



,PTS,TRB,AST,STL,BLK,TOV
3,13.9,9.5,1.6,1.5,1.0,1.7
4,8.9,7.3,2.2,0.9,0.8,1.5
7,21.3,9.2,2.4,0.5,1.3,1.8
10,10.9,8.4,1.4,0.5,1.5,1.3
12,9.4,7.5,1.3,0.8,0.4,0.9


Standardize the features in your dataset using scikit-learn's StandardScaler, which will set the mean of each feature to 0 and the variance to 1.

In [5]:
# answer goes here

scaler = StandardScaler()
nba_scaled = scaler.fit_transform(nba)

nba_scaled = pd.DataFrame(nba_scaled, columns=nba.columns, index=nba.index)
nba_scaled.head()



,PTS,TRB,AST,STL,BLK,TOV
3,0.114638,1.612909,-0.617816,1.393220,0.824001,0.023511
4,-0.744985,0.768627,-0.330886,-0.050271,0.420162,-0.218831
7,1.386880,1.497780,-0.235242,-1.012598,1.429758,0.144681
10,-0.401136,1.190768,-0.713459,-1.012598,1.833596,-0.461172
12,-0.659023,0.845380,-0.761281,-0.290853,-0.387514,-0.945854


Run both K-Means and Gaussian mixtures modeling twice, once with 3 cluster and once with 7 clusters. Print out the resulting means for all 4 scenarios (KM+3, GMM+3, KM+7, GMM+7). When printing the means, transform the scaled versions back into their corresponding unscaled values. 

What "archetypes" do you see? Are the archetypes identified by GMM similar to those identified by K-Means? How do the means of GMM differ from those of K-Means?

In [6]:
# answer goes here
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(nba_scaled)
pd.Series(kmeans.labels_).value_counts()


2    446
1    179
0    179
dtype: int64

In [7]:
centroids = kmeans.cluster_centers_
centroids = scaler.inverse_transform(centroids)
centroids_df = pd.DataFrame(centroids, columns=nba.columns)
centroids_df.style.background_gradient()

,PTS,TRB,AST,STL,BLK,TOV
0,13.763128,8.625698,1.920670,0.775978,1.235196,1.675419
1,19.491620,5.293855,5.934078,1.354190,0.486034,2.797765
2,10.508744,3.962556,2.060762,0.805157,0.376233,1.234305


In [8]:
gauss = GaussianMixture(n_components=3, random_state=42)
gauss.fit(nba_scaled)

GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=100,
                means_init=None, n_components=3, n_init=1, precisions_init=None,
                random_state=42, reg_covar=1e-06, tol=0.001, verbose=0,
                verbose_interval=10, warm_start=False, weights_init=None)

In [9]:
gauss.means_

array([[-0.51467755, -0.31492147, -0.62610129, -0.33759366, -0.33003111,
        -0.69665929],
       [ 0.2090548 ,  1.22199876, -0.30239603, -0.34906286,  1.27208011,
         0.1819726 ],
       [ 0.45824478, -0.42947632,  0.91865828,  0.61602887, -0.44453539,
         0.68580452]])

In [18]:
prob_df = pd.DataFrame(gauss.predict_proba(nba_scaled))
prob_df.shape

(804, 3)

Predict the likelihood of each player belonging to one of the 3 clusters using the GMM model. Then, calculate the entropy for each set of predicted probabilities. 

We will use entropy as a measure of how confident we are in the predicted class label. If we had no confidence in our prediction, we would assign 33% probability to each class, while if we were totally confident, we would assign 100% to one class. Entropy would be at a maximum in the "no confidence" scenario and a minimum in the "full confidence" scenario, which makes it a reasonable way to quantify our uncertainty in our prediction. There are certainly other methods as well; feel free to experiment with them if desired.

Which five predicted labels are we least confident about? Which five are we most confident about? Print out the associated details (season, player name, stats, etc.) from those players.

In [12]:
import scipy 

In [13]:
# answer goes here
scipy.stats.entropy([0.731826, 0.267606, 5.687710e-04])



0.5855042262910504

In [14]:
scipy.stats.entropy([0.57803, 0.419676, 2.286479e-03])


0.6951295395118391

In [15]:
scipy.stats.entropy([0, 1, 0])


0.0

In [16]:
scipy.stats.entropy([0.33 ,0.33, 0.33])


1.0986122886681096

In [20]:
prob_df['entropy'] = prob_df.apply(scipy.stats.entropy, axis=1)

In [21]:
prob_df.sort_values('entropy')

,0,1,2,entropy
797,3.737530e-40,1.000000,1.990853e-48,3.393115e-38
182,7.327602e-23,1.000000,3.252446e-25,3.753057e-21
351,4.964628e-24,1.000000,5.875595e-19,2.466502e-17
252,5.458892e-22,1.000000,6.185078e-19,2.595889e-17
454,2.128166e-32,1.000000,1.912987e-17,7.364098e-16
...,...,...,...,...
109,5.575451e-01,0.091533,3.509223e-01,9.120669e-01
433,5.262444e-01,0.381323,9.243232e-02,9.255872e-01
343,5.552300e-01,0.267582,1.771875e-01,9.860748e-01
741,4.142693e-01,0.205024,3.807062e-01,1.057616e+00
